# Lab 10
Understanding Docker Networks

What is Container Networking?
Container networking also called Software Defined Networking (SDN) is what allows Docker networkgin subsystems to work. 

Some defalt Docker Netwrok drivers are:
- Bridge
    - a Network that is User-defined to allow for all containers connected on the same network to communicate with eachother. 
- Host
    - a container is attahed to a Docker host. Any traffic routed to the container must be routed from the host system.
- Overlay
    - Created in a network spanning multiple docker hosts. Think each of your docker containers in the lab talking to eachother
- Macvlan
    - used in complicated networking to assing logical addresses based on MAC addresses to the virtual network interfaces making it apear as if the container has a physical network interaface connected to the network. Used in legacy applications.
- None
    -  the container only has a loopback interface so it can talk to itself.



# Part 1 - Setup

In [1]:
!docker run -d --name nginx --network=none -p 81:81 nginx

docker: Error response from daemon: Conflict. The container name "/nginx" is already in use by container "44cb21abbddcc409855947856702c86c03e03637a42e77dd95f4a9d1d6b03062". You have to remove (or rename) that container to be able to reuse that name.

Run 'docker run --help' for more information


## Note:
**If you get this error:**

*docker: permission denied while trying to connect to the Docker daemon ...*

run this in wsl: 
``` bash
sudo usermod -aG docker $USER
newgrp docker
sudo systemctl restart docker
```
Run it again....

you should get a hash number YAY!

`cbb2f01cd1b...`

--------------------------------------
If you are still getting the error


run this:
``` bash
sudo chmod 666 /usr/docker.sock
```
--------------------------------------
**If you get this error you did things right**

docker: Error response from daemon: Conflict. The container name "/nginx

--------------------------------------


## we now have an nginx container 

NGINX serves static content (like HTML, CSS, JavaScript, images) efficiently, improving website performance.

In [2]:
!curl localhost

#the curl command should fail on our system 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
curl: (7) Failed to connect to localhost port 80 after 2242 ms: Could not connect to server


Lets run it on the docker container 

In [3]:
!docker exec -it nginx /bin/sh -c "curl google.com"



the input device is not a TTY.  If you are using mintty, try prefixing the command with 'winpty'


* curl:(6) Could not resolve host: google.com

As you can see the nginx container can not talk to google.com

# Part 2 - Understanding the network 
Docker comes with a subcommand for handling Docker networks.

In [4]:
!docker network

Usage:  docker network COMMAND

Manage networks

Commands:
  connect     Connect a container to a network
  create      Create a network
  disconnect  Disconnect a container from a network
  inspect     Display detailed information on one or more networks
  ls          List networks
  prune       Remove all unused networks
  rm          Remove one or more networks

Run 'docker network COMMAND --help' for more information on a command.


running `docker network ls` will show us all the names of the networks.


In [5]:
!docker network ls 

NETWORK ID     NAME      DRIVER    SCOPE
6d33a781e4e4   bridge    bridge    local
0874766683fb   host      host      local
ef5b222716f3   none      null      local


In the above you can see that there is a none network. 

When we setup the docker nginx server we did not give it a network

lets look for the nginx server

In [6]:
!docker network inspect none 

[
    {
        "Name": "none",
        "Id": "ef5b222716f397a8f469d0c7443ca384ed14e51768fc3ec44a2893e30356df66",
        "Created": "2025-04-07T01:48:55.336663273Z",
        "Scope": "local",
        "Driver": "null",
        "EnableIPv4": true,
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": null,
            "Config": null
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "44cb21abbddcc409855947856702c86c03e03637a42e77dd95f4a9d1d6b03062": {
                "Name": "nginx",
                "EndpointID": "1270426ffa1c30e70452e32ca3999310ab8a3fcfcf4d200d8158b807084f9398",
                "MacAddress": "",
                "IPv4Address": "",
                "IPv6Address": ""
            }
        },
        "Options": {},
        "Labels": {}
    }
]


## OK Enough of that nonsense
Lets install docker and use the API to do some work this is a python class after all

In [7]:
!pip install docker

In [8]:
import docker

client = docker.from_env()

print("Docker version:", client.version()['Version'])


Docker version: 28.0.4


c:\Users\jayve\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\jayve\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\jayve\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


Lets create a quick code to list the containers using the docker client

In [9]:
def list_running_containers():
    containers = client.containers.list(all=True)# list all containers
    #containers = client.containers.list() #list only running containers
    for container in containers:
        print(f"{container.name} - {container.status}")

list_running_containers()

nginx - running


Here you can see the running container and no surprise it is the nginx we started before


In [10]:
#lest identify the network of the nginx container
def show_container_ip(container_name):
    container = client.containers.get(container_name)
    networks = container.attrs['NetworkSettings']['Networks']
    
    for net_name, net_data in networks.items():
        print(f"{container_name} in network '{net_name}' has IP: {net_data['IPAddress']}")


show_container_ip("nginx")


nginx in network 'none' has IP: 


again no surprise there is no IP address as it does not have a network. 

### ok I lied back to creating more stuff though the CLI

In [20]:
#setup another container for mysql
!docker run -d --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8
#Note the -d means detached mode so no container is using created just the MySQL database 

4457b2076a6ecf97f69cc875167216552d264f3e6ca060483ad5318cf80ae869


## The following needs to be run in the terminal

In [21]:
!docker run -d --name adminer -p 8080:8080 adminer


docker: Error response from daemon: Conflict. The container name "/adminer" is already in use by container "11eaeb5a4eb5b303320bf7e5964ffb5b07f82df0ff8190da993bf36e19fcc063". You have to remove (or rename) that container to be able to reuse that name.

Run 'docker run --help' for more information


Run the following each in there own terminal

``` bash
docker logs -f mysql
docker logs -f adminer
```

In [22]:
!docker inspect mysql | grep -n1 "IPAddress\":"
# or
print("-"*30)
show_container_ip("mysql")


------------------------------
mysql in network 'bridge' has IP: 172.17.0.3


'grep' is not recognized as an internal or external command,
operable program or batch file.


In [23]:
!docker inspect adminer| grep  -n1 "IPAddress\":"
# or
print("-"*30)
show_container_ip("adminer")


------------------------------
adminer in network 'bridge' has IP: 172.17.0.2


'grep' is not recognized as an internal or external command,
operable program or batch file.


### Lets do a bit of cleanup 

``` bash
#lets remove our old container
!docker stop adminer
!docker rm adminer
!docker stop mysql
!docker rm mysql
!docker stop nginx 
!docker rm nginx
```

In [ ]:
def clean_containers():
    containers = client.containers.list(all=True)
    if not containers:
        print("No containers to clean.")
        return
    
    for container in containers:
        container.stop()
        print(f"Waiting for {container.name} to stop...")
        while container.status != 'exited':
            pass
        
        container.reload()
        if container.status == 'exited':
            print(f"Removing {container.name}")
            container.remove()

list_running_containers()

clean_containers()


mysql - running
adminer - running
nginx - exited
Waiting for mysql to stop...


KeyboardInterrupt: 

: 

## Setting up the network

In [ ]:
# Create a network called yourNetwork using the following command

!docker network create yourNetwork

431d092aa909f50dab25f1d2dc5eef9c97b516314e2bb79519d8198541843e8c


In [ ]:
!docker network inspect yourNetwork

[
    {
        "Name": "yourNetwork",
        "Id": "431d092aa909f50dab25f1d2dc5eef9c97b516314e2bb79519d8198541843e8c",
        "Created": "2025-04-18T14:36:24.265817696-07:00",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": {},
            "Config": [
                {
                    "Subnet": "172.18.0.0/16",
                    "Gateway": "172.18.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {},
        "Options": {},
        "Labels": {}
    }
]


In [ ]:
!docker run -d --network yourNetwork --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8


b5bcdf9fdee61ea1de797aa354e1f332c31ff095d1f2390319ff36f7fc36397b


In [ ]:
!docker run -d --network yourNetwork --name adminer -p 8080:8080 adminer

a96cb2039f9ca306043e6584c324b684156f1fca0b19ae63bf4108895642d665


In [ ]:
#Now lets connect the adminer
!docker network connect yourNetwork adminer

Error response from daemon: endpoint with name adminer already exists in network yourNetwork


In [ ]:
!docker inspect mysql |grep -C2 " \"Networks\""
!docker inspect mysql |grep -A9 " \"Networks\""| grep " \"IPAddress\""
#or
print("-"*30)
show_container_ip("mysql")

            "IPv6Gateway": "",
            "MacAddress": "",
            "Networks": {
                "yourNetwork": {
                    "IPAMConfig": null,
                    "IPAddress": "172.18.0.2",
------------------------------


NameError: name 'show_container_ip' is not defined

In [ ]:
!docker inspect adminer| grep -A2 " \"Networks\""
!docker inspect adminer| grep -A9 " \"Networks\""| grep " \"IPAddress\""
!ping -c 1 172.18.0.3
#or
print("-"*30)
show_container_ip("adminer")

            "Networks": {
                "yourNetwork": {
                    "IPAMConfig": null,
                    "IPAddress": "172.18.0.3",
PING 172.18.0.3 (172.18.0.3) 56(84) bytes of data.
64 bytes from 172.18.0.3: icmp_seq=1 ttl=64 time=0.063 ms

--- 172.18.0.3 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 0.063/0.063/0.063/0.000 ms
------------------------------


NameError: name 'show_container_ip' is not defined

# LAB 
As a reminder you should be submitting to me 2 files. 
- _YourName_Lab 09.ipynb
    - this is your lab report
- _YourName__\<name of your script\>.py

## Part 1 - warm up

Now that you have 2 containers up and running and know there connected. Create a python script that will check to see if the IP addresses of the containers are up and running. 




## Part 2
Continue with the above code and create a script that will:
- Relaunch a container if it is ever shutdown. 
- Perform a maintenance restart after 1 day has passed. 
- Monitor the log files and outputs errors and warnings.

your python file does not need to be in a docker container


https://docs.docker.com/engine/api/sdk/examples/

